In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD functions
from bailey_module_four import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
db = AnimalShelter(user="accuser", password="hackham7")

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
        html.H2("Connor Bailey - Pi In The Sky"),
        html.B(html.H1('CS-340 Dashboard')),
    ]),
    html.Hr(),
    html.Div([
        html.Label("Filter (Rescue Type)"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Tracking', 'value': 'disaster'},
                {'label': "Show All", 'value': 'all'}
            ],
            value='all', # default value
            labelStyle={'display': 'inline-block', 'margin-right': '25px'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single',
                         style_table={'overflowX' : 'auto'}
                        ),
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='piechart-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {
            'breed': {
                '$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
            },
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'mountain':
        query = {
            'breed': {
                '$in': [
                    'German Shepherd',
                    'Alaskan Malamute',
                    'Old English Sheepdog',
                    'Siberian Husky',
                    'Rottweiler'
                ]
            },
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'disaster':
        query = {
            'breed': {
                '$in': [
                    'Doberman Pinscher', 
                    'German Shepard',
                    'Golden Retriever',
                    'Bloodhound',
                    'Rottweiler'
                ]
            },
            'sex_upcon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    else: # 'all' is selected
        query = {}
    
    # Pass query to MongoDB
    results = db.read(query)
    df = pd.DataFrame.from_records(results)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('piechart-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return dcc.Graph(figure=px.pie(names=[], values=[], title="No Data Found"))
    
    # Create the DataFrame
    dff = pd.DataFrame(viewData)
    
    # Group the data
    pie_data = dff['breed'].value_counts().reset_index()
    pie_data.columns = ['breed', 'count']
    
    # Build and return the Pie Chart
    chart = px.pie(
        pie_data,
        names='breed',
        values='count',
        title='Different Breeds'
    )
    chart.update_traces(textinfo='percent+label')
    
    return dcc.Graph(figure=chart)
    
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None or len(index) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connecting with user=accuser, password=hackham7, host=nv-desktop-services.apporto.com, port=33205, db=AAC
Dash app running on http://127.0.0.1:18236/


/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

